In [ ]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import average_precision_score

from src import Dataset
from src.dataset import download_dataset, process_dataset
from src.nn import YoloDataset
from src.analysis import plot_sample_images, print_dataset_summary

In [ ]:
# Настройка путей
datasets_dir = Path("data/raw")
OUTPUT_DIR = Path("data/processed")
ANNOTATIONS_DIR = OUTPUT_DIR / "annotations"
IMAGES_DIR = OUTPUT_DIR / "images"


# Создание директорий
ANNOTATIONS_DIR.mkdir(parents=True, exist_ok=True)
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

# Определение датасетов
datasets = [
    Dataset(
        name="dasmehdixtr",
        base_path=datasets_dir / "dasmehdixtr" / "drone_dataset_yolo" / "dataset_txt"
    ),
    Dataset(
        name="dasmehdixtr",
        meta_type="xml",
        base_path=datasets_dir / "dasmehdixtr" / "dataset_xml_format" / "dataset_xml_format"
    ),
    Dataset(
        name="mcagriaksoy",
        base_path=datasets_dir / "mcagriaksoy" / "Database1" / "Database1"
    ),
]

In [ ]:
# Загрузка и обработка датасетов
for dataset in datasets:
    download_dataset(dataset, datasets_dir)
    process_dataset(dataset, IMAGES_DIR, ANNOTATIONS_DIR)
    print(f"Processing Dataset {dataset.name}_{dataset.meta_type}...")
    print_dataset_summary(dataset)
    plot_sample_images(dataset, num_samples=3)

# Проверка результатов
image_files = list(IMAGES_DIR.iterdir())
annotation_files = list(ANNOTATIONS_DIR.iterdir())

In [ ]:
# Загрузка и обработка датасетов
for dataset in datasets:
    download_dataset(dataset, datasets_dir)
    process_dataset(dataset, IMAGES_DIR, ANNOTATIONS_DIR)
    print(f"Processing Dataset {dataset.name}_{dataset.meta_type}...")
    print_dataset_summary(dataset)
    plot_sample_images(dataset, num_samples=3)

# Проверка результатов
image_files = list(IMAGES_DIR.iterdir())
annotation_files = list(ANNOTATIONS_DIR.iterdir())

print(f"\nВсего изображений: {len(image_files)}")
print(f"\nВсего аннотаций: {len(annotation_files)}")

# Анализ объединенного датасета
combined_dataset = Dataset(
    name="combined",
    base_path=OUTPUT_DIR
)

print("\nАнализ объединенного датасета:")
print("-" * 50)

# Получаем статистику по датасету
stats = print_dataset_summary(combined_dataset)

# Визуализируем примеры изображений с разметкой
print("\nПримеры изображений с разметкой:")
plot_sample_images(combined_dataset, num_samples=5)

print("\nСтатистика по датасету:")
print(f"Среднее количество объектов на изображение: {stats['avg_boxes_per_image']:.2f}")
print(f"Процент изображений с дронами: {(stats['images_with_drones'] / stats['total_images'] * 100):.1f}%")

# Проверяем баланс классов и размеры объектов
print("\nПроверка завершена. Можно приступать к обучению модели.")

# Baseline Model

In [ ]:
# Подготовка данных для обучения
transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = YoloDataset(str(IMAGES_DIR), str(ANNOTATIONS_DIR), transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
# Настройка модели
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Настройка классификатора
num_classes = 2  # фон + дрон
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.to(device)

# Оптимизатор и планировщик
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
def train(model, optimizer, data_loader, device):
    model.train()
    total_loss = 0
    for images, targets in tqdm(data_loader, desc="Training"):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()
    
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    aps = []
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            outputs = model(images)

            for target, output in zip(targets, outputs):
                gt_boxes = target['boxes'].cpu().numpy()
                pred_boxes = output['boxes'].cpu().numpy()
                pred_scores = output['scores'].cpu().numpy()
                if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                    aps.append(average_precision_score(gt_boxes.flatten(), pred_boxes.flatten()))
    
    return np.mean(aps) if aps else 0.0

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss = train(model, optimizer, train_loader, device)
    val_ap = evaluate(model, val_loader, device)
    
    lr_scheduler.step()
    print(f"Loss: {train_loss:.4f}, Val mAP: {val_ap:.4f}")

# Сохранение модели
torch.save(model.state_dict(), "models/faster_rcnn_drone_detector.pth")